In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
output_path = "/data/ephemeral/home/movie/zerovirus/movierec/output/"
catboost = pd.read_csv(os.path.join(output_path, "CatBoost_output.csv"))
ease = pd.read_csv(os.path.join(output_path, "EASE_output.csv"))
easer = pd.read_csv(os.path.join(output_path, "EASER_output.csv"))
multivae = pd.read_csv(os.path.join(output_path, "multiVAE_output.csv"))
s3rec = pd.read_csv(os.path.join(output_path, "S3Rec_output.csv"))

In [3]:
print(f"catboost shape: {catboost.shape}")
print(f"ease shape: {ease.shape}") 
print(f"easer shape: {easer.shape}") 
print(f"multivae shape: {multivae.shape}") 
print(f"s3rec shape: {s3rec.shape}") 

catboost shape: (313600, 2)
ease shape: (313600, 2)
easer shape: (313600, 2)
multivae shape: (313600, 2)
s3rec shape: (313600, 2)


In [4]:
from tqdm import tqdm
import numpy as np

np.random.seed(42)

p = [8, 2]
outputs = pd.concat([
    easer,
    multivae["item"],
], axis=1)

# # 열 방향으로 샘플링하여 새 열 생성
# outputs['Sampled'] =outputs.apply(lambda row: np.random.choice(row[1:], p=p), axis=1)
def ensemble_models(outputs, p):
    output_group = outputs.groupby("user")
    sampled = pd.DataFrame(columns=["user", "item"])
    for user_id, group in tqdm(output_group, desc="Sampling..."):
        group = group.iloc[:, 1:]
        for i in range(len(p)):
            for _ in range(p[i]-1):
                group = pd.concat([group, group.iloc[:, i]], axis=1)
        items = group.values.flatten()

        lst = []
        for _ in range(10):
            r = np.random.choice(items)
            lst.append(r)
            items = items[items!=r]
        
        result = pd.DataFrame({"user": user_id, "item": lst})
        sampled = pd.concat([sampled, result], axis=0)

    return sampled

sampled_output = ensemble_models(outputs, p)

Sampling...: 100%|██████████| 31360/31360 [05:15<00:00, 99.43it/s] 


In [5]:
sampled_output

,user,item
0,11,59784
1,11,33004
2,11,4886
3,11,3156
4,11,2
...,...,...
5,138493,32587
6,138493,53125
7,138493,58559
8,138493,2762


In [6]:
user_group = sampled_output.groupby("user")["item"].nunique()
user_group.value_counts()

item
10    31360
Name: count, dtype: int64

In [7]:
sampled_output.to_csv("ensemble_output.csv", index=False)